In [1]:
import re
from collections import Counter
import numpy as np

def load_and_clean_text(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()

    # Remove Gutenberg header
    start_marker = "*** START OF"
    end_marker = "*** END OF"
    
    start_idx = text.find(start_marker)
    end_idx = text.find(end_marker)
    
    if start_idx != -1 and end_idx != -1:
        text = text[start_idx:end_idx]

    # Lowercase
    text = text.lower()

    # Keep only letters and spaces
    text = re.sub(r"[^a-z\s]", " ", text)

    # Remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

def tokenize(text):
    return text.split()

def build_vocab(tokens, min_count=2):
    word_counts = Counter(tokens)

    # Remove rare words
    vocab_words = [word for word, count in word_counts.items() if count >= min_count]

    word_to_idx = {word: idx for idx, word in enumerate(vocab_words)}
    idx_to_word = {idx: word for word, idx in word_to_idx.items()}

    return word_to_idx, idx_to_word

def tokens_to_indices(tokens, word_to_idx):
    return [word_to_idx[word] for word in tokens if word in word_to_idx]

def build_negative_sampling_dist(tokens, word_to_idx):
    word_counts = Counter(tokens)

    vocab_size = len(word_to_idx)
    freqs = np.zeros(vocab_size)

    for word, idx in word_to_idx.items():
        freqs[idx] = word_counts[word]

    # Apply 3/4 power
    freqs = freqs ** 0.75
    freqs /= np.sum(freqs)

    return freqs

def preprocess(filepath, min_count=2):
    text = load_and_clean_text(filepath)
    tokens = tokenize(text)

    word_to_idx, idx_to_word = build_vocab(tokens, min_count)
    corpus_indices = tokens_to_indices(tokens, word_to_idx)

    neg_sampling_dist = build_negative_sampling_dist(tokens, word_to_idx)

    return corpus_indices, word_to_idx, idx_to_word, neg_sampling_dist

In [2]:
corpus, word_to_idx, idx_to_word, neg_dist = preprocess("alice.txt")
print(f"Corpus length (in tokens): {len(corpus)}")
print(f"Vocabulary size: {len(word_to_idx)}")

Corpus length (in tokens): 9382
Vocabulary size: 816


In [3]:
# Save corpus to file
corpus_words = [idx_to_word[idx] for idx in corpus]
corpus_text = " ".join(corpus_words)

with open("corpus_output.txt", "w", encoding="utf-8") as f:
    f.write(corpus_text)

print(f"Corpus saved to 'corpus_output.txt'")
print(f"File preview (first 200 chars): {corpus_text[:200]}")

Corpus saved to 'corpus_output.txt'
File preview (first 200 chars): of the alice s adventures in wonderland illustration alice in the room of the duchess the alice s adventures in wonderland sam l gabriel sons company new york by sam l gabriel sons company new york al


In [4]:
def generate_cbow_samples(corpus_indices, window_size=2):
    """
    Generates CBOW training samples.

    Parameters:
        corpus_indices (list): List of word indices
        window_size (int): Context window size (c)

    Returns:
        samples (list of tuples):
            Each element is (context_indices, target_index)
    """
    
    samples = []
    T = len(corpus_indices)

    for t in range(window_size, T - window_size):
        
        # Target word w_O
        target = corpus_indices[t]

        # Context words w_1 ... w_C
        context = []

        # Left context
        for i in range(t - window_size, t):
            context.append(corpus_indices[i])

        # Right context
        for i in range(t + 1, t + window_size + 1):
            context.append(corpus_indices[i])

        samples.append((context, target))

    return samples


In [5]:
window_size = 2
training_samples = generate_cbow_samples(corpus, window_size)

print("Number of training samples:", len(training_samples))
print("Example sample:", training_samples[0])

Number of training samples: 9378
Example sample: ([0, 1, 3, 4], 2)


In [6]:
with open("training_samples.txt", "w", encoding="utf-8") as f:
    for i, (context, target) in enumerate(training_samples):
        # Convert indices to words
        context_words = [idx_to_word[idx] for idx in context]
        target_word = idx_to_word[target]
        
        # Format: context_words | target_word
        line = f"{' '.join(context_words)} | {target_word}\n"
        f.write(line)

print(f"Training samples saved to 'training_samples.txt' ({len(training_samples)} samples)")

Training samples saved to 'training_samples.txt' (9378 samples)


In [7]:
# Sanity check: verify all context sizes are exactly 4 words
context_sizes = [len(context) for context, target in training_samples]
unique_sizes = set(context_sizes)

print(f"Unique context sizes: {unique_sizes}")
print(f"All contexts are size 4: {unique_sizes == {4}}")
print(f"Min size: {min(context_sizes)}, Max size: {max(context_sizes)}")

Unique context sizes: {4}
All contexts are size 4: True
Min size: 4, Max size: 4


In [8]:
#parameter setup
V = len(word_to_idx)
N = 50

W = np.random.randn(V, N) * 0.01
W_prime = np.random.randn(V, N) * 0.01

In [9]:
# negative sampling setup
def sample_negative_words(K, neg_dist):
    return np.random.choice(len(neg_dist), size=K, p=neg_dist)

In [10]:
def train_cbow(training_samples, W, W_prime, neg_dist, 
               window_size=2, N=50, K=5, lr=0.025, num_epochs=10):
    """
    Train CBOW with Negative Sampling in pure NumPy.
    
    Parameters:
        training_samples : list of tuples
            Each tuple: (context_indices, target_index)
        W : np.ndarray
            Input embedding matrix (V, N)
        W_prime : np.ndarray
            Output embedding matrix (V, N)
        neg_dist : np.ndarray
            Negative sampling distribution (length V)
        window_size : int
            Context window size
        N : int
            Embedding dimension
        K : int
            Number of negative samples
        lr : float
            Learning rate
        num_epochs : int
            Number of training epochs

    Returns:
        W, W_prime : np.ndarrays
            Updated embedding matrices
    """
    
    V = W.shape[0]
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    def sample_negative_words(K, neg_dist):
        return np.random.choice(len(neg_dist), size=K, p=neg_dist)
    
    for epoch in range(num_epochs):
        total_loss = 0.0
        
        for context_indices, target in training_samples:
            
            # Forward Pass 
            v_context = W[context_indices]                 # (C, N)
            h = np.mean(v_context, axis=0)                # (N,)
            
            u_target = W_prime[target]                     # (N,)
            neg_indices = sample_negative_words(K, neg_dist)
            u_neg = W_prime[neg_indices]                  # (K, N)
            
            # Scores
            s_pos = np.dot(u_target, h)
            s_neg = np.dot(u_neg, h)                      # (K,)
            
            # Sigmoid
            sig_pos = sigmoid(s_pos)
            sig_neg = sigmoid(-s_neg)
            
            # Loss 
            loss = -np.log(sig_pos) - np.sum(np.log(sig_neg))
            total_loss += loss
            
            # Backward Pass 
            delta_pos = sig_pos - 1                        # scalar
            delta_neg = 1 - sig_neg                        # (K,)
            
            # Gradients for output embeddings
            grad_u_target = delta_pos * h                  # (N,)
            grad_u_neg = delta_neg[:, None] * h           # (K, N)
            
            # Gradient wrt hidden layer
            grad_h = delta_pos * u_target + np.dot(delta_neg, u_neg)
            
            # Distribute to context embeddings
            grad_v_context = grad_h / len(context_indices)
            
            # Parameter Updates 
            W_prime[target] -= lr * grad_u_target
            W_prime[neg_indices] -= lr * grad_u_neg
            
            for idx in context_indices:
                W[idx] -= lr * grad_v_context
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")
    
    return W, W_prime

In [11]:
# Hyperparameters
N = 50
K = 5
lr = 0.025
num_epochs = 20

# Train embeddings
W, W_prime = train_cbow(training_samples, W, W_prime, neg_dist,
                        window_size=2, N=N, K=K, lr=lr, num_epochs=num_epochs)

Epoch 1/20, Loss: 38965.8890
Epoch 2/20, Loss: 36325.7317
Epoch 3/20, Loss: 30572.7817
Epoch 4/20, Loss: 28093.6809
Epoch 5/20, Loss: 27104.9220
Epoch 6/20, Loss: 26510.1067
Epoch 7/20, Loss: 26027.2008
Epoch 8/20, Loss: 25607.9288
Epoch 9/20, Loss: 25211.3196
Epoch 10/20, Loss: 24814.3068
Epoch 11/20, Loss: 24440.9758
Epoch 12/20, Loss: 24031.3123
Epoch 13/20, Loss: 23649.7937
Epoch 14/20, Loss: 23264.2788
Epoch 15/20, Loss: 22913.9350
Epoch 16/20, Loss: 22518.2563
Epoch 17/20, Loss: 22174.7185
Epoch 18/20, Loss: 21864.0546
Epoch 19/20, Loss: 21520.1256
Epoch 20/20, Loss: 21175.5014


In [12]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [13]:
alice_idx = word_to_idx['alice']
queen_idx = word_to_idx['queen']
rabbit_idx = word_to_idx['rabbit']

v_alice = W[alice_idx]
v_queen = W[queen_idx]
v_rabbit = W[rabbit_idx]

print("alice <-> queen:", cosine_similarity(v_alice, v_queen))
print("alice <-> rabbit:", cosine_similarity(v_alice, v_rabbit))

alice <-> queen: 0.5106595509137505
alice <-> rabbit: 0.5198968419112165


In [14]:
# Generate 20 cosine similarity examples for verification
import random

def cosine_similarity(u, v):
    """Compute cosine similarity between two vectors."""
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    if norm_u == 0 or norm_v == 0:
        return 0
    return dot_product / (norm_u * norm_v)

# Get a sample of words from vocabulary
sample_words = random.sample(list(word_to_idx.keys()), min(20, len(word_to_idx)))
sample_indices = [word_to_idx[w] for w in sample_words]

print("=" * 70)
print("20 COSINE SIMILARITY EXAMPLES FOR VERIFICATION")
print("=" * 70)

examples = []
for i in range(min(20, len(sample_indices) - 1)):
    idx1 = sample_indices[i]
    idx2 = sample_indices[i + 1]
    
    word1 = idx_to_word[idx1]
    word2 = idx_to_word[idx2]
    
    v1 = W[idx1]
    v2 = W[idx2]
    
    similarity = cosine_similarity(v1, v2)
    examples.append((word1, word2, similarity))
    
    print(f"{i+1:2d}. {word1:15s} <-> {word2:15s} : {similarity:7.4f}")

print("=" * 70)

20 COSINE SIMILARITY EXAMPLES FOR VERIFICATION
 1. fur             <-> adventures      :  0.9043
 2. adventures      <-> without         :  0.8447
 3. without         <-> so              :  0.8765
 4. so              <-> it              :  0.7649
 5. it              <-> will            :  0.4264
 6. will            <-> notice          :  0.9169
 7. notice          <-> hardly          :  0.6140
 8. hardly          <-> shorter         :  0.8250
 9. shorter         <-> jurymen         :  0.6567
10. jurymen         <-> under           :  0.8435
11. under           <-> leave           :  0.7832
12. leave           <-> sure            :  0.4329
13. sure            <-> a               :  0.2630
14. a               <-> pig             :  0.1510
15. pig             <-> beg             :  0.1937
16. beg             <-> end             :  0.4024
17. end             <-> moment          :  0.8566
18. moment          <-> felt            :  0.9411
19. felt            <-> most            :  0.7211
